New type test - a custom type with its own validators added to the schema

In [1]:
from collections import namedtuple

import cerberus

In [2]:
Dimensions = namedtuple("Dimensions", "length, width, height")

In [3]:
dimensions_type = cerberus.TypeDefinition('dimensions', (Dimensions,), ())

class Validator(cerberus.Validator):
    types_mapping = cerberus.Validator.types_mapping.copy()
    types_mapping['dimensions'] = dimensions_type
    
    def _validate_max_largest_dimension(self, constraint, field, value):
        """ {"type": "number", "min": 0} """
        if max(value) > constraint:
            self._error(field, f"longest dimension is larger than the limit of {constraint}")
    
    def _validate_max_volume(self, constraint, field, value):
        """ {"type": "number", "min": 0}"""
        volume = value.length * value.width * value.height
        if volume > constraint:
            self._error(field, f"total volume is greater than the maximum of {constraint}")

In [4]:
schema = {
    "size": {
        "type": "dimensions",
        "max largest dimension": 10,
        "max volume": 250
    }
}
validator = Validator(schema)

In [5]:
assert validator.validate({"size": Dimensions(8, 5, 6)})

In [6]:
assert not validator.validate({"size": Dimensions(11, 0.5, 0.3)})
validator.errors

{'size': ['longest dimension is larger than the limit of 10']}

In [7]:
assert not validator.validate({"size": Dimensions(8, 5, 7)})
validator.errors

{'size': ['total volume is greater than the maximum of 250']}